In [3]:
import numpy as np

## First order low-pass filter

### Continuous transfer function
The low-pass filter transfer function is
$$H(s) = \frac{\omega_0}{s + \omega_0}$$

### Discrete transfer function
Computing the discrete transfer function using Tustin's method (bilinear transform), set $s = \frac{2}{\Delta t} \left( \frac{1-z^{-1}}{1+z^{-1}} \right)$, so
\begin{align*} 
H(z) &= \frac{\omega_0}{\frac{2}{\Delta t} \frac{1-z^{-1}}{1+z^{-1}}  + \omega_0} \\
     &= \frac{ \Delta t \omega_0 + ( \Delta t \omega_0 ) z^{-1} }{ (\Delta t \omega_0 + 2) + (\Delta t \omega_0 - 2) z^{-1} }
 \end{align*}


We want to find the filter coefficients for the discrete update:
$$y[n] = a_1 y[n-1] + a_2 y[n-2] + ... + b_0 x[n] + b_1 x[n-1] + ...$$

The coefficients can be taken directly from the discrete transfer function of the filter in the form:
$$H(z) = \frac{b_0 + b_1 z^{-1} + b_2 z^{-2} + \ldots}{1 - a_1 z^{-1} - a_2 z^{-2} + \ldots}$$

Setting $\alpha = \omega_0 \Delta t$, the transfer function is
\begin{align*} 
H(z) &= \frac{ \alpha+ \alpha z^{-1} }{ (\alpha + 2) + (\alpha - 2) z^{-1} } \\
     &= \frac{ \frac{\alpha}{\alpha+2} + \frac{\alpha}{\alpha + 2} z^{-1} }{ 1 + \frac{\alpha - 2}{\alpha + 2} z^{-1} }
 \end{align*}


$$ a_1 = - \frac{\alpha - 2}{\alpha + 2}\quad  \text{and} \quad b_0 = b_1 = \frac{\alpha}{\alpha + 2}   $$

### Arduino code
On the arduino code, we compute the coefficients as

<pre>
float alpha = omega0*dt;
a[0] = -(alpha - 2.0)/(alpha+2.0);
b[0] = alpha/(alpha+2.0);
b[1] = alpha/(alpha+2.0);        
</pre>

In [12]:
# Example coefficients for testing
dt = 1/1.0e3;
omega0 = 5*2*np.pi;
alpha = omega0*dt;
a = [0];
b = [0,0];
a[0] = -(alpha - 2.0)/(alpha+2.0);
b[0] = alpha/(alpha+2.0);
b[1] = alpha/(alpha+2.0); 

print(b)
print(a)

[0.015465039003346996, 0.015465039003346996]
[0.9690699219933061]


<hr>

## Second order low-pass Butterworth filter

### Continuous transfer function

The $n^{\text{th}}$ Butterworth low-pass filter transfer function with $\omega_c = 1$ can be written as (see https://en.wikipedia.org/wiki/Butterworth_filter)
$$H(s) = \frac{1}{\sum_0^{n} \beta_k s^k}$$
where $n$ is the order of the filter. The coefficients are given by the recursion formula:
$$\beta_{k+1} = \frac{\cos( k \gamma )}{\sin((k+1)\gamma)} \beta_k$$
with $\beta_0 = 1$ and $\gamma = \frac{\pi}{2n}$.

The $n^{\text{th}}$ order Butterworth polynomial with a cutoff frequency $\omega_c$ can be written as
$$B_n(s) = \sum_{k=0}^n c_k s^k$$ 
with $c_k = \frac{\beta_k}{{\omega_c}^k}$

This implies that a second order filter is:
$$H(s) = \frac{1}{ c_0 + c_1 s + c_2 s^2}$$ 
with $c_k = \frac{\beta_k}{{\omega_c}^k}$. Using the recursion formula with $n = 2$, the Butterworth coefficients are:
$$\beta_0 = 1 \qquad \beta_1 = \sqrt{2} \qquad \beta_2 =  1$$

### Discrete transfer function

Computing the discrete transfer function using Tustin's method (bilinear transform), set $s = \frac{2}{\Delta t} \left( \frac{1-z^{-1}}{1+z^{-1}} \right)$, so (with $\alpha = \omega_c \Delta t$)

\begin{align*} 
H(z) &=\frac{1}{ c_0 + c_1 \left[ \frac{2}{\Delta t} \left( \frac{1-z^{-1}}{1+z^{-1}} \right) \right] + c_2 \left[ \frac{2}{\Delta t} \left( \frac{1-z^{-1}}{1+z^{-1}} \right) \right]^2} \\
     &=\frac{1}{ \beta_0 + 2 \beta_1 \frac{1}{\omega_c \Delta t} \left( \frac{1-z^{-1}}{1+z^{-1}} \right)+ 4 \beta_2 \frac{1}{\omega_c^2 \Delta t^2} \left( \frac{1-z^{-1}}{1+z^{-1}} \right)^2} \\
     &=\frac{1}{ \beta_0 + 2 \beta_1 \frac{1}{\alpha} \left( \frac{1-z^{-1}}{1+z^{-1}} \right)+ 4 \beta_2 \frac{1}{\alpha^2} \left( \frac{1-z^{-1}}{1+z^{-1}} \right)^2} \\  
     &=\frac{\alpha^2 + 2 \alpha^2 z^{-1} + \alpha^2 z^{-2}}{ (\alpha^2 \beta_0 + 2 \alpha \beta_1 + 4 \beta_2) + (2 \alpha^2 \beta_0 - 8 \beta_2) z^{-1} + (\beta_0 \alpha^2 - 2 \beta_1 \alpha + 4 \beta_2) z^{-2}} 
 \end{align*}
Setting $D = \alpha^2 \beta_0 + 2 \alpha \beta_1 + 4 \beta_2$, this is

$$ H(z) =\frac{ (\alpha^2/D) + (2 \alpha^2/D) z^{-1} + (\alpha^2/D) z^{-2}}{ 1 + [(2 \alpha^2 \beta_0 - 8 \beta_2)/D] z^{-1} + [(\beta_0 \alpha^2 - 2 \beta_1 \alpha + 4 \beta_2)/D] z^{-2}}$$


We want to find the filter coefficients for the discrete update:
$$y[n] = a_1 y[n-1] +a_2 y[n-2] + ... + b_0 x[n] + b_1 x[n-1] + ...$$

The coefficients can be taken directly from the discrete transfer function of the filter in the form:
$$H(z) = \frac{b_0 + b_1 z^{-1} + b_2 z^{-2} + \ldots}{1 - a_1 z^{-1} - a_2 z^{-2} + \ldots}$$

So the discrete update coefficients are

$$ b_0 = \alpha^2/D \quad b_1 = 2 \alpha^2/D \quad b_2 = \alpha^2/D $$

$$ a_1 = - (2 \alpha^2 \beta_0 - 8 \beta_2)/D \quad a_2 = -(\beta_0 \alpha^2 - 2 \beta_1 \alpha + 4 \beta_2)/D $$

### Arduino code
On the arduino code, we compute the coefficients as

<pre>
float alpha = omega0*dt;
float alphaSq = alpha*alpha;
float beta[] = {1, sqrt(2), 1};
float D = alphaSq*beta[0] + 2*alpha*beta[1] + 4*beta[2];

b[0] = alphaSq/D;
b[1] = 2*b[0];
b[2] = b[0];

a[0] = -(2*alphaSq*beta[0] - 8*beta[2])/D;
a[1] = -(beta[0]*alphaSq - 2*beta[1]*alpha + 4*beta[2])/D;        
</pre>

In [10]:
# Example coefficients for testing
dt = 1/1.0e3;
omega0 = 5*2*np.pi;
alpha = omega0*dt;
alphaSq = alpha*alpha;
beta= [1,np.sqrt(2), 1];
D = alphaSq*beta[0] + 2*alpha*beta[1] + 4*beta[2];

b = [0,0,0]
b[0] = alphaSq/D;
b[1] = 2*b[0];
b[2] = b[0];

a = [0,0]
a[0] = -(2*alphaSq*beta[0] - 8*beta[2])/D;
a[1] = -(beta[0]*alphaSq - 2*beta[1]*alpha + 4*beta[2])/D;     

print(b)
print(a)

[0.00024131978889242034, 0.0004826395777848407, 0.00024131978889242034]
[1.9555818921741432, -0.956547171329713]
